In [1]:
!pip install -U datasets
!pip install -U accelerate
!pip install -U transformers
!pip install -U huggingface_hub

  Using cached pyarrow-17.0.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (3.3 kB)
  Using cached pandas-2.2.2-cp312-cp312-macosx_11_0_arm64.whl.metadata (19 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached tqdm-4.66.5-py3-none-any.whl.metadata (57 kB)
  Using cached fsspec-2024.6.1-py3-none-any.whl.metadata (11 kB)
  Using cached PyYAML-6.0.2-cp312-cp312-macosx_11_0_arm64.whl.metadata (2.1 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl.metadata (4.0 kB)
  Using cached attrs-24.2.0-py3-none-any.whl.metadata (11 kB)
  Using cached frozenlist-1.4.1-cp312-cp312-macosx_11_0_arm64.whl.metadata (12 kB)
  Using cached charset_normalizer-3.3.2-cp312-cp312-macosx_11_0_arm64.whl.metadata (33 kB)
  Using cached urllib3-2.2.2-py3-none-any.whl.metadata (6.4 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 8.6 MB/s eta 0:00:00
Using cached fsspec-2024.6.1-py3-none-any.whl (177 kB

In [24]:
from datasets import load_dataset

dataset = load_dataset("ilsilfverskiold/tech-keywords-topics-summary")
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'source', 'text', 'timestamp', 'reactions', 'engagement', 'url', 'text_length', 'keywords', 'topic', 'summary', '__index_level_0__'],
        num_rows: 7196
    })
    validation: Dataset({
        features: ['id', 'source', 'text', 'timestamp', 'reactions', 'engagement', 'url', 'text_length', 'keywords', 'topic', 'summary', '__index_level_0__'],
        num_rows: 635
    })
    test: Dataset({
        features: ['id', 'source', 'text', 'timestamp', 'reactions', 'engagement', 'url', 'text_length', 'keywords', 'topic', 'summary', '__index_level_0__'],
        num_rows: 635
    })
})

In [30]:
dataset['train']['text']

['appsmith - Platform to build admin panels, internal tools, and dashboards. Integrates with 15+ databases and any API. - Organizations build internal applications such as dashboards, database GUIs, admin panels, approval apps, customer support dashboards, and more to help their teams perform day-to-day operations. Appsmith is an open-source tool that enables the rapid development of these internal apps. Read more on our website.InstallationThere are two ways to start using Appsmith:Signup on Appsmith Cloud.Ins',
 'Elon Musk went to camp where kids were encouraged to bully others for food/water',
 'Symfony Event System, Understanding and Implementing Event-driven Architecture - Explore real-world use cases and best practices for efficient software development',
 'Introduction To Swift Programming : Swift Cheat Sheet - A comprehensive Guide on Classes, Protocols, Extensions, Enums, Initializers, and Deinitializers',
 '10 Things You Didn’t Know About Famous Programmers Like Bill Gates an

In [5]:
def show_samples(dataset, num_samples=3, seed=42):
    sample = dataset["train"].shuffle(seed=seed).select(range(num_samples))
    for example in sample:
        print(f"\n'>> Text: {example['text']}'")
        print(f"'>> Keywords: {example['keywords']}'")


show_samples(dataset)


'>> Text: Driverless car users will not be prosecuted for fatal crashes in UK'
'>> Keywords: Driverless Cars, Legal Issues, UK'

'>> Text: Google is embedding inaudible watermarks right into its AI generated music -'
'>> Keywords: Google, AI Music, Watermarks, Audio Technology'

'>> Text: What are your thoughts on Nextjs performance? Do you agree with this chart? - ( by 10up where Nextjs appears lower than WordPress on core vitals. Couldn’t post the image here due to community rules. But appreciate any other studies and thought you have on this matter.'
'>> Keywords: Next.js, Performance, 10up, WordPress'


In [4]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_name = 'facebook/bart-large' # go smaller if you can
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

texts = dataset['train']['text']

# Tokenize all texts and find the maximum length (max for BART is 1024 tokens)
max_token_length = max(len(tokenizer.encode(text, truncation=True)) for text in texts)
print(f"The longest text is {max_token_length} tokens long.")

/opt/miniconda3/envs/plp_module/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


The longest text is 245 tokens long.


In [5]:
def get_feature(batch):
  encodings = tokenizer(batch['text'], text_target=batch['keywords'],
                        max_length=1024, truncation=True)

  encodings = {'input_ids': encodings['input_ids'],
               'attention_mask': encodings['attention_mask'],
               'labels': encodings['labels']}

  return encodings

dataset_pt = dataset.map(get_feature, batched=True)
dataset_pt

Map: 100%|██████████| 635/635 [00:00<00:00, 22430.94 examples/s]


DatasetDict({
    train: Dataset({
        features: ['id', 'source', 'text', 'timestamp', 'reactions', 'engagement', 'url', 'text_length', 'keywords', 'topic', 'summary', '__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 7196
    })
    validation: Dataset({
        features: ['id', 'source', 'text', 'timestamp', 'reactions', 'engagement', 'url', 'text_length', 'keywords', 'topic', 'summary', '__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 635
    })
    test: Dataset({
        features: ['id', 'source', 'text', 'timestamp', 'reactions', 'engagement', 'url', 'text_length', 'keywords', 'topic', 'summary', '__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 635
    })
})

In [6]:
columns = ['input_ids', 'labels', 'attention_mask']
dataset_pt.set_format(type='torch', columns=columns)

In [7]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [8]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir = 'bart_tech_keywords', # rename to what you want it to be called
    num_train_epochs=3, # your choice
    warmup_steps = 500,
    per_device_train_batch_size=4, 
    per_device_eval_batch_size=4,
    weight_decay = 0.01,
    logging_steps = 10,
    evaluation_strategy = 'steps',
    eval_steps=50, 
    save_steps=1e6,
    gradient_accumulation_steps=16 
)

trainer = Trainer(model=model, args=training_args, tokenizer=tokenizer, data_collator=data_collator,
                  train_dataset = dataset_pt['train'], eval_dataset = dataset_pt['validation'])

trainer.train()

/opt/miniconda3/envs/plp_module/lib/python3.12/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
  0%|          | 0/336 [00:00<?, ?it/s]/opt/miniconda3/envs/plp_module/lib/python3.12/site-packages/transformers/data/data_collator.py:656: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:281.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)
  3%|▎         | 10/336 [01:20<37:17,  6.86s/it] 

{'loss': 3.6513, 'grad_norm': 40.85301971435547, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.09}


  6%|▌         | 20/336 [02:41<45:16,  8.60s/it]

{'loss': 3.1626, 'grad_norm': 31.181283950805664, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.18}


  9%|▉         | 30/336 [03:56<35:56,  7.05s/it]

...

 98%|█████████▊| 330/336 [2:13:27<01:21, 13.51s/it]

{'loss': 0.8061, 'grad_norm': 10.157153129577637, 'learning_rate': 3.3e-05, 'epoch': 2.93}


100%|██████████| 336/336 [2:14:41<00:00, 12.53s/it]Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
100%|██████████| 336/336 [2:14:45<00:00, 24.07s/it]

{'train_runtime': 8085.979, 'train_samples_per_second': 2.67, 'train_steps_per_second': 0.042, 'train_loss': 1.1810229207788194, 'epoch': 2.99}


TrainOutput(global_step=336, training_loss=1.1810229207788194, metrics={'train_runtime': 8085.979, 'train_samples_per_second': 2.67, 'train_steps_per_second': 0.042, 'total_flos': 3582300094562304.0, 'train_loss': 1.1810229207788194, 'epoch': 2.9883268482490273})

In [16]:
trainer.save_model('tech-keywords-extractor_plp')


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


In [70]:
from bert_score import score
predictions = ["hello world", "general kenobi"]
references = ["goodnight moon", "the sun is shining"]
results = score(predictions, references, model_type="distilbert-base-uncased")
print(results)

/opt/miniconda3/envs/plp_module/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


(tensor([0.7900, 0.5584]), tensor([0.7900, 0.5889]), tensor([0.7900, 0.5732]))


In [8]:
# !huggingface-cli login --token hf_xxxxxxxxxxxx

# check output of fine tuned model
from transformers import pipeline

pipe = pipeline('summarization', model='tech-keywords-extractor_plp')

text_test = "Apple is launching iphone 16 with Apple Intelligence. It also uses OpenAI ChatGPT"

# test_text=dataset['test'][0]['text']
# keywords = dataset['test'][0]['keywords']
print("the text: ", text_test)
print("generated keywords: ", pipe(text_test))
# print("orginal keywords : ",keywords)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Your max_length is set to 128, but your input_length is only 21. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=10)


the text:  Apple is launching iphone 16 with Apple Intelligence. It also uses OpenAI ChatGPT
generated keywords:  [{'summary_text': 'Apple, iphone 16, Apple Intelligence, OpenAI, ChatGPT'}]


In [29]:
# Check back on output of base untrained model

pipe = pipeline('summarization', model = 'facebook/bart-large')

text_test = "Apple is launching iphone 16 with Apple Intelligence. It also uses OpenAI ChatGPT. "

# test_text=dataset['test'][0]['text']
# keywords = dataset['test'][0]['keywords']
print("the text: ", text_test)
print("generated keywords: ", pipe(text_test))
# print("orginal keywords : ",keywords)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Your max_length is set to 128, but your input_length is only 23. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=11)


the text:  Apple is launching iphone 16 with Apple Intelligence. It also uses OpenAI ChatGPT. 
generated keywords:  [{'summary_text': 'Apple is launching iphone 16 with Apple Intelligence. It also uses OpenAI ChatGPT. '}]


In [30]:
# Check back on output of base model fine tuned on CNN daily news

pipe = pipeline('summarization', model = 'facebook/bart-large-cnn')

text_test = "Apple is launching iphone 16 with Apple Intelligence. It also uses OpenAI ChatGPT. "

# test_text=dataset['test'][0]['text']
# keywords = dataset['test'][0]['keywords']
print("the text: ", text_test)
print("generated keywords: ", pipe(text_test))
# print("orginal keywords : ",keywords)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Your max_length is set to 142, but your input_length is only 23. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=11)


the text:  Apple is launching iphone 16 with Apple Intelligence. It also uses OpenAI ChatGPT. 
generated keywords:  [{'summary_text': "Apple is launching iphone 16 with Apple Intelligence. It also uses OpenAI ChatGPT. Apple is launching Apple Intelligence with Apple Artificial Intelligence on the iPhone 16. Apple  also using OpenAI chatGPT on the phone. Apple's new iPhone 16 will be released on September 18."}]


In [ ]:
for i in range(0, 50):
    text_test = dataset['test'][i]['text']
    keywords = dataset['test'][i]['keywords']
    print("text: ", text_test)
    print("generated keywords: ", pipe(text_test)[0]['summary_text'])
    print("original keywords: ", keywords)

In [9]:
pipe(dataset['test'][1]['text'])[0]['summary_text']

Your max_length is set to 128, but your input_length is only 12. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=6)


'Distractions, analytical thinking, Fake News'

In [10]:
dataset['test'][1]['keywords']

'Distractions, Analytical Thinking, Fake News'

In [24]:
# you would replace your own name here
# you do not need to create a repository beforehand
trainer.push_to_hub("wbcmthh42/tech-keywords-extractor_plp")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
training_args.bin:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

training_args.bin: 100%|██████████| 5.18k/5.18k [00:00<00:00, 18.3kB/s]






































































































































































































































































































































































































































CommitInfo(commit_url='https://huggingface.co/wbcmthh42/bart_tech_keywords/commit/aaa476e530d52c5ef3c58dd4029c1cdf966f6d6c', commit_message='wbcmthh42/tech-keywords-extractor_plp', commit_description='', oid='aaa476e530d52c5ef3c58dd4029c1cdf966f6d6c', pr_url=None, pr_revision=None, pr_num=None)

In [11]:
!pip install bert-score

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached matplotlib-3.9.2-cp312-cp312-macosx_11_0_arm64.whl.metadata (11 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.53.1-cp312-cp312-macosx_11_0_arm64.whl.metadata (162 kB)
  Using cached pillow-10.4.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (9.2 kB)
Using cached matplotlib-3.9.2-cp312-cp312-macosx_11_0_arm64.whl (7.8 MB)
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
Using cached fonttools-4.53.1-cp312-cp312-macosx_11_0_arm64.whl (2.2 MB)
Using cached pillow-10.4.0-cp312-cp312-macosx_11_0_arm64.whl (3.4 MB)


In [12]:
!bert-score-show --lang en -r "There are two bananas on the table." -c "On the table are two apples." -f out.png

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Matplotlib is building the font cache; this may take a moment.
tokenizer_config.json: 100%|█████████████████| 25.0/25.0 [00:00<00:00, 67.6kB/s]
config.json: 100%|█████████████████████████████| 482/482 [00:00<00:00, 1.85MB/s]
vocab.json: 100%|████████████████████████████| 899k/899k [00:00<00:00, 5.91MB/s]
merges.txt: 100%|████████████████████████████| 456k/456k [00:00<00:00, 2.95MB/s]
tokenizer.json: 100%|██████████████████████| 1.36M/1.36M [00:00<00:00, 8.56MB/s]
/opt/miniconda3/envs/plp_module/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
model.safetensors: 100%|███████████████████| 1.42G/1.42G [00:52<00:00, 27.1MB/s]
Some weights of RobertaModel were not in

In [20]:
from bert_score import score

P, R, F1 = score(pipe(dataset['test'][0]['text'])[0]['summary_text'], dataset['test'][1]['keywords'], lang="en", verbose=True)

Your max_length is set to 128, but your input_length is only 12. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=6)
/opt/miniconda3/envs/plp_module/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


100%|██████████| 1/1 [00:00<00:00,  9.27it/s]


computing greedy matching.


100%|██████████| 1/1 [00:00<00:00, 499.68it/s]

done in 0.11 seconds, 394.60 sentences/sec


In [48]:
P, R, F1 = score(pipe(dataset['test'][1]['text'])[0]['summary_text'], dataset['test'][1]['keywords'], lang="en", verbose=True)

Your max_length is set to 128, but your input_length is only 12. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=6)
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


100%|██████████| 1/1 [00:00<00:00,  8.64it/s]


computing greedy matching.


100%|██████████| 1/1 [00:00<00:00, 534.58it/s]

done in 0.12 seconds, 368.70 sentences/sec


44

In [41]:
dataset['test'][0]['text']

'Unleash the Edge: Women’s Faux Leather Zip-Up Motorcycle Jacket\u200a—\u200aVintage Grunge Aesthetic Revived - Ride the Rebel Wave: Women’s Faux Leather Zip-Up Motorcycle Jacket\u200a—\u200aA Fusion of Timeless Edge and Vintage Grunge Aesthetic for…'

In [60]:
pipe(dataset['test'][1]['text'])[0]['summary_text']

Your max_length is set to 128, but your input_length is only 12. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=6)


'Distractions, analytical thinking, Fake News'

In [61]:
dataset['test'][1]['keywords']

'Distractions, Analytical Thinking, Fake News'

In [64]:
summary = pipe(dataset['test'][3]['text'])[0]['summary_text']
ground_truth = dataset['test'][3]['keywords']
P, R, F1 = score([summary], [ground_truth], lang="en", verbose=True)

Your max_length is set to 128, but your input_length is only 36. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=18)
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


100%|██████████| 1/1 [00:00<00:00,  9.24it/s]


computing greedy matching.


100%|██████████| 1/1 [00:00<00:00, 680.34it/s]

done in 0.11 seconds, 8.96 sentences/sec


In [65]:
print(f"System level F1 score: {F1.mean():.3f}")
print(f"System level P score: {P.mean():.3f}")
print(f"System level R score: {R.mean():.3f}")

System level F1 score: 0.963
System level P score: 0.952
System level R score: 0.974


In [79]:
import pandas as pd

results = pd.read_csv('/Users/tayjohnny/Documents/My_MTECH/PLP/bart_tech_keywords_results.csv', header=0)
average_P, average_R, average_F1 = results[['P', 'R', 'F1']].mean()

print(f"System level average P score: {average_P:.3f}")
print(f"System level average R score: {average_R:.3f}")
print(f"System level average F1 score: {average_F1:.3f}")

System level average F1 score: 0.942
System level average P score: 0.939
System level average R score: 0.945
